### Astronomical Source Detection using UDF  f105w image

#### Following  the  tutorial  found  at https://sep.readthedocs.io/en/v1.0.x/tutorial.html,  but  using  the astropy  fits  routines  instead  of fitsio

#### Importing libraries such as numpy astropy.io.fits instead of fitsio and matplotlib.pyplot <br> *(by Veronika C. Joseph)*

In [ ]:
import numpy as np
import sep
import matplotlib.pyplot as plt
from astropy.io import fits 
from astropy import units
from astropy.utils.data import download_file
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

#### Read in image from a FITS file usitng astropy.io fits.open method   <br> *(by Veronika C. Joseph)*

In [ ]:
# hdulist returns an object HDUList that is list like collection of HDU objects
#
# *** I am using a local file (instead of downloading file directly) because there seems to
#     be a problem with SSL certification validation using my current version of python.
#     You will need to paste in your custom local path to image.fits if you want to test this.

hdulist = fits.open('/Users/veronikajoseph/Documents/UCSC-Classes/ASTR_119/final_project/data/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')

# trying the download_file feature -- doesn't work due to SSL certiicate verification problem
#image_file = download_file('https://github.com/kbarbary/sep/blob/v1.0.x/data/image.fits', cache=True)

# -- below shows info about the file we are reading-in
print('Image info:\n')
hdulist.info()

# image_data stores primary info about our FITS file
image_data = hdulist[0].data 

# close fits file since we stored all the info into variable
hdulist.close()


#### Display the image from FITS file that was read in using plt.imshow method and plt.colorbar() <br> *(by Veronika C. Joseph)*

In [ ]:
# m - stores mean of our image_data 
# s - stores standard deviation of image_data 
# plt.imshow(..) displays an image data on 2D regular raster
#      cmap - is a colormap instance that maps scalar data to colors
#      vmin / vmax - defines the data range that the colormap covers 
#      origin - places [0,0] index of the array in the lower left corner of axes

m, s = np.mean(image_data), np.std(image_data)

plt.imshow(image_data, interpolation='nearest', cmap='bone', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

# saving fits file without object detection marks
plt.savefig('f105w_v1.0_drz.png')

##### Plotting using LogNorm to get better idea of objects on this image

In [ ]:
plt.imshow(image_data, cmap='RdBu_r', norm=LogNorm())

cbar = plt.colorbar(ticks=[5.e3, 1.e3, 2.e4])
cbar.ax.set_yticklabels(['5,000', '10,000', '20,000'])

 #### Subtracting background using sep.bckground method, which will return Background object that holds the spatially varying background and spatially varying background noise level <br> *(by Veronika C. Joseph)*

In [ ]:
image_data = image_data.byteswap(inplace=True).newbyteorder()

bkg = sep.Background(image_data, bw=64, bh=64, fw=3, fh=3)

#### Obtaining various information about our image from Background object bkg by using SEP methods such as globalback, globalrms <br> *(by Veronika C. Joseph)*

In [ ]:
# get a "global" mean and noise of the image background:
print('Global background level: ', bkg.globalback)
print('Global background RMS:   ', bkg.globalrms)

In [ ]:
# evaluate background as 2-D array, same size as original image
bkg_image = bkg.back()

# bkg_image = np.array(bkg)   # equivalent to above

#### Showing our 2D background array using plt.imshow method specifying various attributes of graph  <br> *(by Veronika C. Joseph)*

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray',origin='lower')
plt.colorbar();

In [ ]:
# evaluate the background noise as 2-D array, same size as original image
bkg_rms = bkg.rms()

#### Similar process here but instead we are showing 2D background noise array  <br> *(by Veronika C. Joseph)*

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();

##### Subtracting background from our image and saving the result in data_sub variable <br>*(by Veronika C. Joseph)*

In [ ]:
# subtract the background 
data_sub = image_data - bkg

#### Running object detection on the background-subtracted data using sep.extract method that extracts sources from an image by specifying various controlling detection tresholds <br> *(by Veronika C. Joseph)*

In [ ]:
# data_sub - is our new image data with subtracted background 
# 1.5 - is our current treshold pixel value for detection 
# err - specifies pixel-by-pixel detection treshold 

# testing various thresholds to try to eliminate noise 
#objects = sep.extract(data_sub, 30, err=bkg.globalrms)
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms, mask=image_data<=0.03)

In [ ]:
# how many objects were detected
print('Number of objects detected: ', len(objects))

#### Using objects['x'] and objects['y'] we'll obtain the centroid coordinates of the objects that were detected above.  Using Ellipse imported from matplotlib.patches, we'll overplot the object coordinates with basic shape parameters to check for the detected objects on our FITS image using custom plotting attributes <br>*(by Veronika C. Joseph)*

In [ ]:
from matplotlib.patches import Ellipse
from matplotlib.colors import LogNorm

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='bone', vmin=m-s, vmax=m+s, origin='lower')
#plt.imshow(image_data, cmap='bone', norm=LogNorm())

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy = (objects['x'][i], objects['y'][i]),
               width  = 6*objects['a'][i],
               height = 6*objects['b'][i],
               angle  = objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

    
plt.savefig('f105w_with_circled_objects.png', bbox_inches="tight", dpi=600)

#### Next, we are performing simple circular aperture photometry with a 3.0 pixel radius at location of the objects using sep.sum_circle method that sums data in circular aperture(s) <br>*(by Veronika C. Joseph)*

In [ ]:
# flux, fluxerr, flag: are all 1-D array with one entry per object
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))  